Item 35 Avoid Causing State Transitions in Generators with throw

Things to Remember
- The throw method can be used to re-raise exceptions within generators at the position of the most recently executed yield expression
- Using throw harms readability because it requires additional nesting and boilerplate in order to raise and catch exceptions
- A better way to provide exceptional behavior in generators is to use a class that implements the \__iter\__ method along with methods to cause exceptional state transitions  

In [ ]:
# understand how throw work
class MyError(Exception):
    pass

def my_generator():
    yield 1
    # - re-raise the exception after the output 2 is generated
    #   and the generator is resumed 
    yield 2
    yield 3 # will not be reached

it = my_generator()
print(next(it)) # 1
print(next(it)) # 2
print(it.throw(MyError('test error'))) # error


Understand the throw method
- when the throw method is called, the last occurrence of a yield expression re-raise the provided Exception instance after its output is received instead of continuing normally 
- in the example above the last occurrence of a yield expression is "yield 2"

In [ ]:
# try/expect compound statement in generators
def my_generator():
    yield 1
    try:
        yield 2
    except MyError:
        print('Got MyError!')
    else:
        yield 3
    yield 4

it = my_generator()
print(next(it)) 
print(next(it))
# - this provides a two-way communication channel
#   between a generator and its caller
print(it.throw(MyError('test error'))) # print Got MyError first and then 4

In [ ]:
# write a timer that supports sporadic resets

# - no ideal to declare a global
# - but serve the purpose in this
#   simple example 
global TimesCalled
TimesCalled = 0

# - the idea is that we are polling for exteranl reset
# - in our case we simply it to return True only
#   if it has been called 4 times  
def check_for_reset():
    global TimesCalled # how to access a global variable
    TimesCalled += 1
    if TimesCalled == 4:
        return True
    return False

class Reset(Exception):
    pass

def announce(remaining):
    print(f'{remaining} ticks remaining')


In [ ]:
TimesCalled = 0
def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            print(f'Reset {current} to {period}')
            current = period # reset to the original period

def run():
    it = timer(4)
    while True:
        try:
            if check_for_reset():
                # - notify the generator to reset the timer
                current = it.throw(Reset()) 
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)

run()   

Problems with the above approach
- the code is much harder to reader than necessary
- The code is noisy as various levels of nesting is required to catch StopIteration exceptions or decide to throw, call next, or announce.

In [ ]:
# solution - define a stateful closure (item 38) using an iterable container object (item 31)

TimesCalled = 0

class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period
    def reset(self):
        self.current = self.period
    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current

def run():
    timer = Timer(4)
    for current in timer:
        if check_for_reset():
            timer.reset()
            print(f'timer current: {timer.current}')
        # - you are announcing the 'current' 
        #   before the timer got reset
        # - as a result, the outputs are slightly different
        #   from the "throw" method approach where 
        #   in this approach the ticks will reach 0 after
        #   the timer reset and then goes back to 3 as 
        #   opposed to in the "throw" method approach where 
        #   the ticks are back to 3 (never reaches 0) after 
        #   the reset 
        announce(current)
run()    